In [1]:
import pickle
import numpy as np
from CrossSecPlotter import *
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import Matern
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

RANDSTATE = 42

In [2]:
class MyDataset():
    def __init__(self, train_X, valid_X, train_y, valid_y, X_test = None):
        self.X_train = train_X
        self.y_train = train_y
        self.X_valid = valid_X
        self.y_valid = valid_y
        self.X_test = X_test
        
        self.normalized = False
        
    def __str__(self):
        return str((self.train_X.shape, self.train_y.shape, self.valid_X.shape, self.valid_y.shape))

    def normalize(self):
        if self.normalized:
            print("already normalized!")
            return
        
        std_list = np.std(self.X_train, axis=0)
        std_list = np.where(std_list == 0, 1, std_list) # prevent zero division error
        avg_list = np.mean(self.X_train, axis=0)
        self.X_train = (self.X_train - avg_list)/std_list
        if not self.X_valid is None: self.X_valid = (self.X_valid - avg_list)/std_list
        if not self.X_test is None: self.X_test = (self.X_test - avg_list)/std_list
            
        self.std_list = std_list
        self.avg_list = avg_list
        print("dataset normalized!")
        self.normalized = True
        return
    
    def normalize_vec(self, X):
        if not self.normalized:
            print("Dataset never normalized")
        return (X - self.avg_list)/self.std_list
    
    def add_fict(self):
        add_ones = lambda X: np.hstack((X,np.ones((X.shape[0],1))))
        self.X_train = add_ones(self.X_train)
        self.X_valid = add_ones(self.X_valid)
        self.X_test = add_ones(self.X_test)
        print("ficticious dimension added!")
        
    def print_dim(self):
        print("X_train has shape:",self.X_train.shape)
        print("y_train has shape:",self.y_train.shape)
        print("X_valid has shape:",self.X_valid.shape)
        print("y_valid has shape:",self.y_valid.shape)
        if self.X_test: print("X_test has shape:",self.X_test.shape)

def plot_acc(xl, yl, zl, filename, title):
    plt.plot(xl, yl, label="validation")
    plt.plot(xl, zl, label="training")
    plt.title(title)
    plt.xlabel("number of training data points")
    plt.ylabel("accuracy")
    plt.legend()
    plt.savefig(filename)
    plt.show()

In [3]:
def loadXy(data_dir):
    with open(data_dir, "rb") as fp:   # Unpickling
        data_list = pickle.load(fp)

    xpts, ypts, zpts, _, magpts = data_list

    # check data
    assert(len(xpts)==len(ypts) and len(xpts)==len(zpts) and len(xpts)==len(magpts))

    # Transform the data into X, y form
    n = len(xpts)
    d = 3
    X = np.array([xpts, ypts, zpts]).T
    y = np.array(magpts)
    return X, y
def load_train(exp_num, withStability, verbose=3, datafolder = "Data/"):
    datafile = "data%d.pkl"%exp_num if not withStability else "data%d(withStability).pkl"%exp_num
    datafile = datafolder + datafile

    X, y = loadXy(datafile)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=41) #!!! RANDSEED
    venus = MyDataset(X_train, X_valid, y_train, y_valid)
    if verbose&1:
        print("run %d data loaded!"%exp_num)
        venus.print_dim()
        venus.normalize()
    
    alpha = 0.15 # best value is 0.15
    kernel = Matern(nu=0.66)
    # kernel = RBF() #

    gpr = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=alpha).fit(venus.X_train, venus.y_train)

    # check
    if verbose&2:
        print("Experiment %d  "%exp_num)
        print("With Stability:", withStability,"  ")
        print("Train score: %f  "%gpr.score(venus.X_train, venus.y_train))
        print("Test score:  %f  "%gpr.score(venus.X_valid, venus.y_valid))
    
    return venus, gpr, X, y

In [10]:
# # train a model for all data
# for exp_num in range(1,3):
#     for withStability in [True, False]:
#         load_train(exp_num, withStability, verbose=2)

In [4]:
exp_num = 1
withStability = True

venus, gpr, X, y = load_train(exp_num, withStability)
alpha = 0.15


run 1 data loaded!
X_train has shape: (271, 3)
y_train has shape: (271,)
X_valid has shape: (117, 3)
y_valid has shape: (117,)
dataset normalized!
Experiment 1  
With Stability: True   
Train score: 0.969555  
Test score:  0.762494  


In [5]:
# Matern nu tuning
# cv_score = []
# nus = np.linspace(0.4, 2.6, 3)
# relu = lambda x: x if x>0 else 0
# for nu in nus:
#     kernel = Matern(nu=nu)
#     gpr = GaussianProcessRegressor(kernel=kernel,random_state=0, alpha=alpha).fit(venus.X_train, venus.y_train)
#     cvs = cross_val_score(gpr, X, y)
#     avg_cvs = sum(cvs)/len(cvs)
#     print("%f for nu=%f"%(avg_cvs, nu))
#     cv_score.append(avg_cvs)

# cv_score = [relu(x) for x in cv_score]
# plt.plot(nus, cv_score, label="Cross-Validation Score")
# plt.legend()
# plt.xlabel("nu")
# plt.title("Matern Kernel (alpha = 0.15)")
# plt.savefig("Matern nu.png")

In [6]:
# use all available data for the final model
kernel = Matern(nu=0.66)


venus_whole = MyDataset(X, None, y, None)
venus_whole.normalize()

gpr = GaussianProcessRegressor(kernel=kernel,random_state=1, alpha=alpha).fit(venus_whole.X_train, venus_whole.y_train)
print("Whole dataset training score: %f"%gpr.score(venus_whole.X_train, venus_whole.y_train))

dataset normalized!
Whole dataset training score: 0.972226


In [7]:
# save the model and print the normalizing lists used
with open("Models/gprMatern0.66.dump" , "wb") as f:
    pickle.dump(gpr, f)
print(venus_whole.avg_list, venus_whole.std_list)

[102.52640418 104.08696283 122.59401936] [3.34305675 3.45653727 3.00584971]


In [ ]:
def denormalize_gpr(gpr, dataset):
    """Denormalize and unpack gpr into a mean function and a variance function. """
    def gauss_mean(x, y, z):
        arr = np.array([[x,y,z]])
        arr = dataset.normalize_vec(arr)
        result = gpr.predict(arr)
        return result
    
    def gauss_var(x, y, z):
        arr = np.array([[x,y,z]])
        arr = dataset.normalize_vec(arr)
        result = gpr.predict(arr, return_std=True)[1]
        return result
    return gauss_mean, gauss_var

In [8]:
gauss_mean, gauss_var = denormalize_gpr(gpr, venus_whole)

z_values = np.linspace(117, 130, 27)
plot_4D(gauss_mean, z_values, "Figures/GPRplots/mean_refact/", color_num = 21, wbounds=(0, 170))
# plot_4D(gauss_var, z_values, "Figures/GPRplots/var_refact/", color_num = 21, wbounds=(0, 1))

'Cross Sections.gif'

In [9]:
# plot_4D_CS(gauss_mean, 126, wbounds=(0, 170))
plot_4D(gauss_mean, z_values, func_name="test",save_images=False, color_num = 21, wbounds=(0, 170))

'test.gif.gif'

# Compare the Normalized Results of the two runs

In [46]:
# train two models

def train_model_on_run(data_dir):
    X, y = loadXy(data_dir)
    venus_whole = MyDataset(X, None, y, None)
    venus_whole.normalize()

    kernel = Matern(nu=0.66)
    gpr = GaussianProcessRegressor(kernel=kernel,random_state=1, alpha=alpha).fit(venus_whole.X_train, venus_whole.y_train)
    print("Whole dataset training score: %f"%gpr.score(venus_whole.X_train, venus_whole.y_train))
    return gpr, venus_whole

gpr1, venus1 = train_model_on_run("Data/data1.pkl")
gpr2, venus2 = train_model_on_run("Data/data2.pkl")

dataset normalized!
Whole dataset training score: 0.972426
dataset normalized!
Whole dataset training score: 0.956383


In [47]:
# for z in np.linspace(117, 130, 27):
#     plot_cross_section(gpr1, z, dataset=venus1, plotVar=True)
#     plot_cross_section(gpr1, z, dataset=venus1, plotVar=False)

In [48]:
filenames1 = []
filenames2 = []
for z in np.linspace(117, 130, 27):
    filenames1.append(plot_cross_section(gpr2, z, dataset=venus2, plotVar=True))
    filenames2.append(plot_cross_section(gpr2, z, dataset=venus2, plotVar=False, z_ratio=0.87)) # , 
    

with imageio.get_writer('mygif.gif', mode='I') as writer:
    for filename in filenames1:
        image = imageio.imread(filename)
        writer.append_data(image)